In [1]:
# importing libraries 
import pandas as pd
import torch
import random
from tqdm import tqdm
from torch.utils.data import DataLoader
import itertools
from local_transformers.src.transformers import (
    TrainingArguments,
    Trainer
) # reads the local_transformers folder instead of the actual transformers package
# importing libraries 
from local_transformers.src.transformers.models.biogpt.tokenization_biogpt import BioGptTokenizer # reads the local_transformers folder instead of the actual transformers package
from local_transformers.src.transformers.models.CustomBioGPT import CustomBioGptForCausalLM # reads the local_transformers folder instead of the actual transformers package
from local_transformers.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import os
from sklearn.model_selection import KFold
if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')
# read the data
df=pd.read_csv("df.csv")

df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["PNA"] = df["PNA"].astype(int)
df["post_aki_status"] = df["post_aki_status"].astype(int)

# prepare the data
# VERY IMPORTANT: MAKE SURE TO USE CustomBioGptForCausalLM SO WE CAN ACCEPT THE LABELS AS PART OF OUR FINE-TUNNING 
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
class preparing(torch.utils.data.Dataset):
    def __init__(self, notes, additional_labels, tokenizer, max_length=128):
        self.notes = notes
        self.additional_labels = additional_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        note = self.notes[idx]
        additional_label = self.additional_labels[idx]

        # Tokenize the input note
        encoding = self.tokenizer(note, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        # Split the input sequence into input and label sequences
        input_ids = encoding['input_ids'][0][:-1]
        label_ids = encoding['input_ids'][0][1:]

        # Return the input and label sequences along with the additional_label
        return {
            'input_ids': input_ids,
            'attention_mask': encoding['attention_mask'][0][:-1],
            'labels': label_ids,
            'additional_labels': additional_label # HERE WE INCLUDE THE ADDITIONAL_LABELS TO BE CONSIDERED AS PART OF OUR TRAINING
        }

    def __len__(self):
        return len(self.notes)



/home/charles/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-09-08 00:04:31.743193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 00:04:32.206188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
len(df)

84875

In [3]:
# fine-tune the data based on the given specified parameters
def get_model(name, train_dataset,val_dataset):
    model = CustomBioGptForCausalLM.from_pretrained("microsoft/biogpt", output_hidden_states=True, lambda_constant=10)
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,      # total number of training epochs
        per_device_train_batch_size=32,  # batch size per device during training
        per_device_eval_batch_size=32,   # batch size for evaluation
        warmup_steps=1000,          # number of warmup steps for learning rate scheduler
        weight_decay=0.001,          # strength of weight decay
        logging_dir=None,               # Do not store logs
        logging_steps=1000,         # Effectively disables logging
        save_strategy="steps",
        learning_rate=0.000005)

    
    # Create the Trainer object
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        lambda_constant=10)

    # Fine-tune BioGPT on the training dataset
    trainer.train()
    trainer.model.save_pretrained(name)    

In [4]:
# get the embeddings of the specified tuned model. 
def get_biogpt_updated(texts, name):
    batch_size=64
    max_length=128
    texts = list(texts)

    model = CustomBioGptForCausalLM.from_pretrained(name, output_hidden_states=True, lambda_constant=10)  # name of your newly trained gpt embeddings
    tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")


    # Move the model to the GPU if available
    if torch.cuda.is_available():
        model = model.to("cuda")
    # tokenize the input text in batches
    all_embeddings = []
    for i in (range(0, len(texts), batch_size)):
        print("Progress", (i/len(texts))*100)
        batch_texts = texts[i:i + batch_size]
        batch_encoded = tokenizer(batch_texts, padding='max_length', truncation=True, return_tensors="pt",
                                  max_length=max_length)
        input_ids = batch_encoded["input_ids"]
        attention_mask = batch_encoded["attention_mask"]

        # Move the tensors to the GPU if available
        if torch.cuda.is_available():
            input_ids = input_ids.to("cuda")
            attention_mask = attention_mask.to("cuda")

        # process each batch separately
        batch_embeddings = []
        for j in (range(input_ids.shape[0])):
            with torch.no_grad():
                outputs = model(input_ids[j:j + 1], attention_mask=attention_mask[j:j + 1])
                last_hidden_state = outputs.hidden_states[-1]
                masked_hidden_state = last_hidden_state * attention_mask[j:j + 1].unsqueeze(-1)
                embedding = torch.mean(masked_hidden_state, dim=1)
                embedding = F.normalize(embedding, p=2, dim=1)
            
            # If you used GPU, move the embeddings back to CPU for further operations (like converting to numpy array)
            if torch.cuda.is_available():
                embedding = embedding.to("cpu")
            batch_embeddings.append(embedding)

        # concatenate the embeddings from all examples in the batch
        embeddings = torch.cat(batch_embeddings, dim=0)
        all_embeddings.append(embeddings)

    # concatenate the embeddings from all batches
    embeddings = torch.cat(all_embeddings, dim=0)

    # convert the embeddings to a numpy array
    X = embeddings.detach().numpy()

    return (X)




In [5]:
def K_fold_val(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # Define the parameter grid
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df)):
        # Split the data into train/test sets
        train, test = df.iloc[train_index], df.iloc[test_index]
        # Create a new column for embeddings based on the training set only
        name=f"{outcome_col}_fold_{i}"
        i=i+1
        train_new=((train.sample(frac=7/8,random_state=42)))
        val_new = train[~train.index.isin(train_new.index)]
        train_notes=(train_new.reset_index(drop=True))
        val_notes=(val_new.reset_index(drop=True))
        tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
        train_dataset = preparing(train_notes["AN_PROC_NAME"],train_notes[outcome_col], tokenizer)
        val_dataset = preparing(val_notes["AN_PROC_NAME"],val_notes[outcome_col], tokenizer)
        if os.path.isdir(name):
            print("model in folder")
        else:
            get_model(name, train_dataset,val_dataset)
        if os.path.isfile(f'{name}_{outcome_col}.pickle'):
            # print("file present")
            with open(f'{name}_{outcome_col}.pickle', 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_biogpt_updated(train["AN_PROC_NAME"], name)
            X_test = get_biogpt_updated(test["AN_PROC_NAME"], name)

        
            with open(f'{name}_{outcome_col}.pickle', 'wb') as f:
                pickle.dump([X_train,X_test], f)


        # Prepare training and testing data
        y_train = train[outcome_col]
        y_test = test[outcome_col]

        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")

In [6]:
results=K_fold_val("death_in_30", df)

0it [00:00, ?it/s]

model in folder
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.2s


In [ ]:
print(results)

metrics: Mean AUROC: 0.8735580982456976, SD AUROC: 0.0025831157334622414, Mean AUPRC: 0.18368927212493574, SD AUPRC: 0.02331906571459559


## Part II: DVT

In [ ]:
results=K_fold_val("DVT", df)

0it [00:00, ?it/s]Some weights of CustomBioGptForCausalLM were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,4.857900
2000,0.818800
3000,0.741600
4000,0.708200
5000,0.732600


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

...

test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,5.400400
2000,0.738000
3000,0.714500
4000,0.693400
5000,0.726600


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

5it [12:52:25, 9269.11s/it]


In [ ]:
print(results)

metrics: Mean AUROC: 0.7920938075271176, SD AUROC: 0.012007535036698479, Mean AUPRC: 0.027415755470729906, SD AUPRC: 0.00429641382369751


## Part III: PE

In [ ]:
results=K_fold_val("PE", df)

0it [00:00, ?it/s]Some weights of CustomBioGptForCausalLM were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,5.216600
2000,0.577100
3000,0.590900
4000,0.556500
5000,0.525400


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

...

test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,5.217700
2000,0.587500
3000,0.579500
4000,0.557500
5000,0.541000


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

5it [8:17:41, 5972.33s/it]


In [ ]:
print(results)

metrics: Mean AUROC: 0.7474938679512715, SD AUROC: 0.0411475652128732, Mean AUPRC: 0.016756839476681277, SD AUPRC: 0.005604168766437049


## Part 4: PNA

In [6]:
results=K_fold_val("PNA", df)


0it [00:00, ?it/s]

model in folder
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   7.7s
[CV] END .learning_rate=0.1, ma

1it [12:18, 738.26s/it]

model in folder
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.2s
[CV] END .learning_rate=0.1, ma

2it [21:13, 619.06s/it]

...

test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,4.656800
2000,0.735700
3000,0.685600
4000,0.679600
5000,0.670000


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

5it [3:20:21, 2404.21s/it]


In [7]:
print(results)

metrics: Mean AUROC: 0.8249025311068046, SD AUROC: 0.014764153774883396, Mean AUPRC: 0.04838566411328342, SD AUPRC: 0.011800013467893833


## Part 5: AKI

In [8]:
results=K_fold_val("post_aki_status", df)

0it [00:00, ?it/s]Some weights of CustomBioGptForCausalLM were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,8.677400
2000,3.614100
3000,3.409200
4000,3.302900
5000,3.210200


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

...

test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,8.709700
2000,3.577400
3000,3.422100
4000,3.295900
5000,3.251600


Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress 3.3932

5it [8:27:19, 6087.96s/it]


In [9]:
print(results)

metrics: Mean AUROC: 0.8428109581814003, SD AUROC: 0.004050489958769224, Mean AUPRC: 0.4957379230862645, SD AUPRC: 0.012887915041649337


## Part 6: delirium

In [10]:
df = df.dropna(subset=['postop_del']).reset_index(drop=True)
df["postop_del"]=df["postop_del"].astype(int)
results=K_fold_val("postop_del", df)

0it [00:00, ?it/s]Some weights of CustomBioGptForCausalLM were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


test


Step,Training Loss


Progress 0.0
Progress 0.6569492917265448
Progress 1.3138985834530896
Progress 1.9708478751796346
Progress 2.6277971669061793
Progress 3.2847464586327244
Progress 3.941695750359269
Progress 4.598645042085813
Progress 5.2555943338123585
Progress 5.912543625538904
Progress 6.569492917265449
Progress 7.226442208991993
Progress 7.883391500718538
Progress 8.540340792445084
Progress 9.197290084171627
Progress 9.854239375898173
Progress 10.511188667624717
Progress 11.168137959351263
Progress 11.825087251077807
Progress 12.482036542804352
Progress 13.138985834530898
Progress 13.795935126257442
Progress 14.452884417983986
Progress 15.109833709710532
Progress 15.766783001437076
Progress 16.42373229316362
Progress 17.08068158489017
Progress 17.737630876616713
Progress 18.394580168343253
Progress 19.0515294600698
Progress 19.708478751796346
Progress 20.36542804352289
Progress 21.022377335249434
Progress 21.679326626975982
Progress 22.336275918702526
Progress 22.99322521042907
Progress 23.6501745021

1it [20:10, 1210.41s/it]Some weights of CustomBioGptForCausalLM were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

test


/home/charles/codes/EPIC project copy/bioGPT tuned/semi-supervised biogpt/without pre_op/local_transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


Progress 0.0
Progress 0.6568818639022889
Progress 1.3137637278045777
Progress 1.9706455917068666
Progress 2.6275274556091555
Progress 3.284409319511444
Progress 3.941291183413733
Progress 4.598173047316021
Progress 5.255054911218311
Progress 5.9119367751206
Progress 6.568818639022888
Progress 7.225700502925177
Progress 7.882582366827466
Progress 8.539464230729754
Progress 9.196346094632043
Progress 9.853227958534333
Progress 10.510109822436622
Progress 11.16699168633891
Progress 11.8238735502412
Progress 12.480755414143488
Progress 13.137637278045776
Progress 13.794519141948063
Progress 14.451401005850354
Progress 15.108282869752642
Progress 15.765164733654933
Progress 16.422046597557223
Progress 17.07892846145951
Progress 17.735810325361797
Progress 18.392692189264086
Progress 19.049574053166378
Progress 19.706455917068666
Progress 20.36333778097095
Progress 21.020219644873244
Progress 21.677101508775532
Progress 22.33398337267782
Progress 22.990865236580106
Progress 23.6477471004824


5it [1:35:03, 1140.72s/it]


In [11]:
print(results)

metrics: Mean AUROC: 0.6975111340770843, SD AUROC: 0.00447714759670866, Mean AUPRC: 0.6716914365544551, SD AUPRC: 0.0036950012847410453


### Getting 95% CI

In [1]:
import scipy.stats as st
import pickle
import numpy as np
for i in ["update_death_in_30.pickle","update_DVT.pickle","update_PE.pickle","update_PNA.pickle","update_post_aki_status.pickle","update_postop_del.pickle"]:
    with open(i, 'rb') as f:
        data=pickle.load(f)

    print(i,"\n")
    print("mean auroc: ", np.mean(data["auroc"]), st.t.interval(alpha=0.95,df=len(data["auroc"])-1,loc=np.mean(data["auroc"]),scale=st.sem(data["auroc"])))
    print("mean auprc: ", np.mean(data["auprc"]), st.t.interval(alpha=0.95,df=len(data["auprc"])-1,loc=np.mean(data["auprc"]), scale=st.sem(data["auprc"])))
    print("\n---------------------\n")

update_death_in_30.pickle 

mean auroc:  0.8735580982456976 (0.8699721587285323, 0.877144037762863)
mean auprc:  0.18368927212493574 (0.1513172191943984, 0.2160613250554731)

---------------------

update_DVT.pickle 

mean auroc:  0.7920938075271176 (0.7754246765880513, 0.8087629384661839)
mean auprc:  0.027415755470729906 (0.021451376905375352, 0.03338013403608446)

---------------------

update_PE.pickle 

mean auroc:  0.7474938679512715 (0.690371889938227, 0.804615845964316)
mean auprc:  0.016756839476681277 (0.008977006006543012, 0.024536672946819542)

---------------------

update_PNA.pickle 

mean auroc:  0.8249025311068046 (0.8044065998664732, 0.8453984623471359)
mean auprc:  0.04838566411328342 (0.03200461929618245, 0.06476670893038439)

---------------------

update_post_aki_status.pickle 

mean auroc:  0.8428109581814003 (0.8371879766715614, 0.8484339396912391)
mean auprc:  0.4957379230862645 (0.4778466287694683, 0.5136292174030607)

---------------------

update_postop_del.p

/tmp/ipykernel_44281/2422348562.py:9: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  print("mean auroc: ", np.mean(data["auroc"]), st.t.interval(alpha=0.95,df=len(data["auroc"])-1,loc=np.mean(data["auroc"]),scale=st.sem(data["auroc"])))
/tmp/ipykernel_44281/2422348562.py:10: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  print("mean auprc: ", np.mean(data["auprc"]), st.t.interval(alpha=0.95,df=len(data["auprc"])-1,loc=np.mean(data["auprc"]), scale=st.sem(data["auprc"])))
